In [1]:
import os
import pyspark
from pyspark.sql import SparkSession
memory = '2g'
pyspark_submit_args = ' --driver-memory ' + memory + ' pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/26 16:14:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/26 16:14:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.registerTempTable('green')

/usr/local/Cellar/apache-spark/3.2.1/libexec/python/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [4]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [5]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

22/02/26 16:14:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 96.51% for 14 writers
22/02/26 16:14:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 90.08% for 15 writers
22/02/26 16:14:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 84.45% for 16 writers
22/02/26 16:14:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 90.08% for 15 writers
22/02/26 16:14:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 96.51% for 14 writers


In [6]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [7]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [8]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

22/02/26 16:14:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 96.51% for 14 writers
22/02/26 16:14:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 90.08% for 15 writers
22/02/26 16:14:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 84.45% for 16 writers
22/02/26 16:14:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 90.08% for 15 writers
22/02/26 16:14:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 96.51% for 14 writers


In [9]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [10]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [11]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [12]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

22/02/26 16:15:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 96.51% for 14 writers
22/02/26 16:15:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 90.08% for 15 writers
22/02/26 16:15:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 84.45% for 16 writers
22/02/26 16:15:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 90.08% for 15 writers
22/02/26 16:15:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 96.51% for 14 writers


In [13]:
df_join = spark.read.parquet('data/report/revenue/total')

In [15]:
df_join.head(5)

[Row(hour=datetime.datetime(2020, 1, 1, 0, 0), zone=10, green_amount=None, green_number_records=None, yellow_amount=42.41, yellow_number_records=2),
 Row(hour=datetime.datetime(2020, 1, 1, 0, 0), zone=12, green_amount=None, green_number_records=None, yellow_amount=107.0, yellow_number_records=6),
 Row(hour=datetime.datetime(2020, 1, 1, 0, 0), zone=25, green_amount=531.0000000000001, green_number_records=26, yellow_amount=324.35, yellow_number_records=16),
 Row(hour=datetime.datetime(2020, 1, 1, 0, 0), zone=82, green_amount=305.1200000000001, green_number_records=28, yellow_amount=29.8, yellow_number_records=1),
 Row(hour=datetime.datetime(2020, 1, 1, 0, 0), zone=100, green_amount=None, green_number_records=None, yellow_amount=473.66999999999996, yellow_number_records=24)]

In [16]:
df_zones = spark.read.parquet('zones/')

In [17]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [62]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')

In [18]:
df_result.head(5)

[Row(hour=datetime.datetime(2020, 1, 1, 0, 0), zone=10, green_amount=None, green_number_records=None, yellow_amount=42.41, yellow_number_records=2, LocationID='10', Borough='Queens', Zone='Baisley Park', service_zone='Boro Zone'),
 Row(hour=datetime.datetime(2020, 1, 1, 0, 0), zone=12, green_amount=None, green_number_records=None, yellow_amount=107.0, yellow_number_records=6, LocationID='12', Borough='Manhattan', Zone='Battery Park', service_zone='Yellow Zone'),
 Row(hour=datetime.datetime(2020, 1, 1, 0, 0), zone=25, green_amount=531.0000000000001, green_number_records=26, yellow_amount=324.35, yellow_number_records=16, LocationID='25', Borough='Brooklyn', Zone='Boerum Hill', service_zone='Boro Zone'),
 Row(hour=datetime.datetime(2020, 1, 1, 0, 0), zone=82, green_amount=305.1200000000001, green_number_records=28, yellow_amount=29.8, yellow_number_records=1, LocationID='82', Borough='Queens', Zone='Elmhurst', service_zone='Boro Zone'),
 Row(hour=datetime.datetime(2020, 1, 1, 0, 0), zone